Hopefully, This script will allow me to load up and fix up the moored data

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw 
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
from scipy import io
import pickle
import cmocean
import json
import f90nml
from collections import OrderedDict
from matplotlib.colors import LogNorm

fs=16
mpl.rc('xtick', labelsize=fs)
mpl.rc('ytick', labelsize=fs)
mpl.rc('legend', fontsize=fs)
mpl.rc('axes', titlesize=fs)
mpl.rc('axes', labelsize=fs)
mpl.rc('figure', titlesize=fs)
mpl.rc('font', size=fs)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')

import warnings
#warnings.filterwarnings('ignore')
from IPython.display import Markdown, display

%matplotlib inline

In [16]:
saveloc='/ocean/kflanaga/MEOPAR/ORCAData'
year=2018
Mooring='Dockton'

In [17]:
df=io.loadmat(f'{saveloc}/Hoodsport.mat')

Alright. I am going to say that Btemp=temperature, Bdepth=depth, Bfluor= chlorophyll fluorescence
Bsal=Salinity

In [18]:
df['Btime'][1]

array([2119.88475405, 2120.01527199,           nan, ..., 7738.51439525,
       7739.0154456 , 7740.01556713])

In [19]:
len(df['Btime'][1])

21974

In [20]:
np.count_nonzero(~np.isnan(df['Btime'][1]))

19199

In [26]:
newData=list(zip(df['Btime'][1],df['Bdepth'][1],df['Btemp'][1],df['Bsal'][1],df['Bfluor'][1]))
columns=['Time','Z','Temperature','Salinity','Chlorophyll']
df2=pd.DataFrame(newData,columns=columns)

In [27]:
UTC=[]
for yd in df2['Time']:
    if np.isnan(yd) == True:
        UTC.append(float("NaN"))
    else:
        start = dt.datetime(1999,12,31)      
        delta = dt.timedelta(yd)     
        offset = start + delta 
        UTC.append(offset)

In [29]:
df2['dtUTC']=UTC
Lat=47.421817
Lon=-123.112583
press=gsw.p_from_z(-1*df2['Z'],Lat)
df2['SA']=gsw.SA_from_SP(df2['Salinity'],press,
                       Lon,Lat)
df2['CT']=gsw.CT_from_t(df2['SA'],df2['Temperature'],press)

In [30]:
df2

,Time,Z,Temperature,Salinity,Chlorophyll,dtUTC,SA,CT
0,2119.884754,2.478400,11.842760,28.817560,NaN,2005-10-19 21:14:02.749997,28.956310,11.958722
1,2120.015272,2.455000,11.678373,29.004173,NaN,2005-10-20 00:21:59.500003,29.143822,11.789294
2,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,2131.884751,2.457510,11.530739,29.527516,NaN,2005-10-31 21:14:02.499998,29.669685,11.630162
4,2132.015726,2.781848,11.533157,30.464067,NaN,2005-11-01 00:22:38.750001,30.610734,11.613944
...,...,...,...,...,...,...,...,...
21969,7736.514421,2.582894,9.051028,26.634174,0.738204,2021-03-06 12:20:46.000003,26.762409,9.179861
21970,7737.514505,2.562979,9.214279,27.302415,0.529442,2021-03-07 12:20:53.250004,27.433867,9.334069
21971,7738.514395,2.586478,9.290576,28.400130,0.681259,2021-03-08 12:20:43.749995,28.536867,9.393057
21972,7739.015446,2.590265,9.173806,27.369057,0.811092,2021-03-09 00:22:14.499996,27.500829,9.292107


In [10]:
##### Saving data as Pickle files to be used in the summary file
#saveloc='/ocean/kflanaga/MEOPAR/mooredData'
#with open(os.path.join(saveloc,f'data_{Mooring}_{year}.pkl'),'wb') as hh:
#    pickle.dump(df,hh)
    
#with open(os.path.join(saveloc,f'daily_data_{Mooring}_{year}.pkl'),'wb') as hh:
#    pickle.dump(df_daily_avg,hh)